## Other Algorisms: LOCI, NMF, ICA

Another algorism, named Localized Optimized Combination of Images (LOCI, see Lafreniere et al. 2007) has been frequently used in high-contrast imaging.

In our course, we have also talked about the Nonnegative Matrix Factorization (NMF) and the Independent Component Analysis (ICA). How do they work on the high-contrast imaging?

The number of papers found about each method about PSF modelling:
![klip](KLIP.png)
![loci](LOCI.png)
![pca](PCA.png)
![ica](ICA.png)
![nmf](NMF.png)

### Localized Optimized Combination of Images (LOCI)

Males (2017) showed that LOCI is equivalent to PCA where all components are used.

In LOCI, the image is divided into pieces of areas. This can also be done when applying PCA.

(The figure is adopted from Lafreniere et al. 2007)
![lociarea](LOCI_subsection.png)

### A Quick Look at the Output of ICA and NMF

We will need SWarp (https://www.astromatic.net/software/swarp) in the data reduction. The installation is a bit annoying... You can skip the next cell if you want (the results are shown later as png images).

In [1]:
### Read the data ###

import os
import numpy as np
from scipy.optimize import minimize
from astropy.io import fits
import matplotlib.pyplot as plt
import glob
import pyklip.instruments.GPI as GPI

from sklearn.decomposition import NMF
from sklearn.decomposition import FastICA
from sklearn.decomposition import RandomizedPCA

cwd=os.getcwd()
#data_dir=cwd+'/../HD10647_J_131117'
data_dir=cwd+'/betaPic_J_131210'

filelist = glob.glob(data_dir+"/*.fits.gz")
dataset = GPI.GPIData(filelist, highpass=True)

Reading File: /home/ymh/astr502/finalproject/Ast-502-final-project/Minghao/betaPic_J_131210/S20131210S0106_spdc_distorcorr.fits.gz
Reading File: /home/ymh/astr502/finalproject/Ast-502-final-project/Minghao/betaPic_J_131210/S20131210S0105_spdc_distorcorr.fits.gz
Reading File: /home/ymh/astr502/finalproject/Ast-502-final-project/Minghao/betaPic_J_131210/S20131210S0097_spdc_distorcorr.fits.gz
Reading File: /home/ymh/astr502/finalproject/Ast-502-final-project/Minghao/betaPic_J_131210/S20131210S0094_spdc_distorcorr.fits.gz
Reading File: /home/ymh/astr502/finalproject/Ast-502-final-project/Minghao/betaPic_J_131210/S20131210S0099_spdc_distorcorr.fits.gz
Reading File: /home/ymh/astr502/finalproject/Ast-502-final-project/Minghao/betaPic_J_131210/S20131210S0107_spdc_distorcorr.fits.gz
Reading File: /home/ymh/astr502/finalproject/Ast-502-final-project/Minghao/betaPic_J_131210/S20131210S0098_spdc_distorcorr.fits.gz
Reading File: /home/ymh/astr502/finalproject/Ast-502-final-project/Minghao/betaPic_

In [98]:
### Preparing the data ###

imagelist=[]
imagecube=dataset.input[1::37]
size=imagecube[0].size

for index in range(len(imagecube)):
    weight=np.sum(imagecube[index])
    imagelist.append(imagecube[index].reshape(size)/weight)
    
imagelist=np.array(imagelist)
imagelist[np.isnan(imagelist)]=0
imagelist[np.isinf(imagelist)]=0

#weight=np.mean(imagelist,axis=1)
#imagelist=imagelist/weight

#print np.min(weight)

imagelist_nmf=imagelist.copy()
imagelist_nmf[imagelist_nmf<0]=0

#weight_nmf=np.mean(imagelist_nmf,axis=1)
#imagelist_nmf=imagelist_nmf/weight_nmf

In [133]:
### Run the algorisms ###

ica = FastICA(n_components=10)
pca=RandomizedPCA(n_components=10)
nmf=NMF(n_components=10)

#ica_result=ica.fit_transform(imagelist)
#pca_result=pca.fit_transform(imagelist)
#nmf_result=nmf.fit_transform(imagelist_nmf)
ica.fit(imagelist)
pca.fit(imagelist)
nmf.fit(imagelist_nmf)

#print len(pca_result)

#ica_components=ica.components_
#pca_components=pca.components_
#nmf_components=nmf.components_

#ica_result=ica.transform(imagelist)
#pca_result=pca.transform(imagelist)
#nmf_result=nmf.transform(imagelist_nmf)

/home/ymh/anaconda2/lib/python2.7/site-packages/sklearn/utils/deprecation.py:52: DeprecationWarning: Class RandomizedPCA is deprecated; RandomizedPCA was deprecated in 0.18 and will be removed in 0.20. Use PCA(svd_solver='randomized') instead. The new implementation DOES NOT store whiten ``components_``. Apply transform to get them.
  warnings.warn(msg, category=DeprecationWarning)


NMF(alpha=0.0, beta=1, eta=0.1, init=None, l1_ratio=0.0, max_iter=200,
  n_components=10, nls_max_iter=2000, random_state=None, shuffle=False,
  solver='cd', sparseness=None, tol=0.0001, verbose=0)

In [150]:
def show_results(datacube=imagecube, method='pca', n_components=10):

    filenamelist=[]
    
    ica = FastICA(n_components=10)
    pca=RandomizedPCA(n_components=10)
    nmf=NMF(n_components=10)

    #print dataset.filenames[::37]
    fitmodel_dict={'pca':pca,\
                     'ica':ica,\
                     'nmf':nmf}

    imagelist_dict={'pca':imagelist,\
                     'ica':imagelist,\
                     'nmf':imagelist_nmf}
    
    fitmodel=fitmodel_dict[method]
    
    imagelist_method=imagelist_dict[m]
    fitmodel.fit(imagelist_method)
    shape=imagecube[0].shape
    
    for raw_index in range(len(imagelist)):
        i=0
        raw_image=imagelist[raw_index]
        raw_image=raw_image.reshape(shape)
        
        if method in ['pca','ica']:
            raw_image=raw_image/np.sum(raw_image)
            raw_image=raw_image-np.mean(raw_image)
        elif method in ['nmf']:
            raw_image[raw_image<0]=0
            raw_image=raw_image/np.sum(raw_image)
            
        raw_row=raw_image.reshape(raw_image.size)

        coeffs=fitmodel.transform([raw_row])
        fitdata_reconstruct=np.dot(coeffs,fitmodel.components_)
        fitdata_reconstruct=fitdata_reconstruct.reshape(shape)
        
        fitsdata=fits.open(dataset.filenames[raw_index*37])
        
        fitsdata[1].data=raw_image-fitdata_reconstruct
        if os.path.exists(data_dir+'/save/tmp'+str(raw_index)+'.fits'):
            os.system('rm '+data_dir+'/save/tmp'+str(raw_index)+'.fits')
        fitsdata.writeto(data_dir+'/save/tmp'+str(raw_index)+'.fits')
        filenamelist.append(data_dir+'/save/tmp'+str(raw_index)+'.fits')
        
    np.savetxt('swarp_input',filenamelist,fmt='%s')
    os.system('swarp @swarp_input -IMAGEOUT_NAME ./coadd'+method+str(n_components)+'.fits')
    
    return 'coadd'+method+str(n_components)+'.fits'
        

In [151]:
show_results(method='ica',n_components=10)
show_results(method='pca',n_components=10)
show_results(method='nmf',n_components=10)

'coaddnmf10.fits'

#### NMF
![nmf](NMF_sub.png)

#### ICA
![ica](ICA_sub.png)

#### PCA
![pca](PCA_sub.png)

### Independent Component Analysis (ICA)

Recall that ICA decompose the signal to several statistically irrelavant components (the crocktail party problem).

How can we intepret this process in PSF modelling?
... I do not know. Probably this is the reason why ICA does not work well.


### Nonnegative Matrix Factorization (NMF)

NMF tries to decompose the signal into several non-negative parts. It is useful when intepreting a signal that is a combination of some non-negative parts.

It is not difficult to intepret NMF in PSF modeling, since PSF can be regarded as a combination of several patterns (for examples, rings of an Airy pattern).

However, when we are trying to MODEL the shape of a PSF, rather than EXPLAIN why the PSF looks like this, NMF is less straightforward than PCA.

## Summary

Anyway, it seems that naively applying ICA or NMF is not a good choice for high-contrast imaging... At least not as good as PCA.